# De Finale Modellen Creeren en hier een verklaring bij geven.
Hierbij zijn voor Thomas en Erwin 2 modellen gemaakt: met en zonder WOX.


Indien WOX data beschikbaar is, gebruiken wij het wox model. Hier wordt uitgegaan van een optimaal iteraties at 75. Voor het niet WOX model wordt een aantal iteraties van X gebruikt. 
Deze waardes zijn om overfitting te voorkomen. Wij willen het model alleen dingen leren die te generaliseren zijn. 


De WOX en niet WOX modellen worden samengevoegd om tot een Ensemble te komen.

Via SHAP values worden verklaringen gegeven voor waarom een voorspelling wordt gedaan. 

Per buurt willen wij hierdoor per emsemble model (= wox/geen wox samengevoegd)
* Voorspelling TK 
* Voorspelling EE 
* Confidence TK voorspellingen 
* Confidence EE voorspellingen
* Feature Importance TK
* Feature Importance EE

---------------
In deze notebook realiseren wij een output voor Powerbi
- samenvoegen van de 4 modellen tot 2 waarbij wox/ geen wox wordt gecombineerd.
- confidence toevoegen
- 5 belangrijkste features meegeven.

In [1]:
import pandas as pd

In [3]:
predictions = pd.read_csv('../2019model_predictions.csv')

In [4]:
predictions.head()

,Unnamed: 0,buurtcode,ee_wox,ee_nowox,TK_wox,TK_nowox
0,0,30000,c,b,c,c
1,1,30001,c,b,c,c
2,2,30002,c,b,c,c
3,3,30007,c,b,c,c
4,4,30008,c,c,c,c


In [5]:
# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"

# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')
buurtendata.rename(columns={"RLBBEV18":"RLBBEV",'KL18': "KL", 
               'RLBRMTR18': 'RLBRMTR', 'RLBWON18':'RLBWON' , 'RLBBEV18':'RLBBEV', 'RLBVRZ18':'RLBVRZ', 'RLBVEI18': 'RLBVEI', 'RLBFYS18': 'RLBFYS',}, inplace=True) 

In [7]:
predictions['wox_info'] = buurtendata['wox'].notnull()
predictions['EE_Ensemble'] = predictions.apply(lambda x: x['ee_wox'] if x['wox_info'] else x['ee_nowox'], axis=1)
predictions['TK_Ensemble'] = predictions.apply(lambda x: x['TK_wox'] if x['wox_info'] else x['TK_nowox'], axis=1)


In [8]:
predictions.head()

,Unnamed: 0,buurtcode,ee_wox,ee_nowox,TK_wox,TK_nowox,wox_info,EE_Ensemble,TK_Ensemble
0,0,30000,c,b,c,c,False,b,c
1,1,30001,c,b,c,c,False,b,c
2,2,30002,c,b,c,c,False,b,c
3,3,30007,c,b,c,c,False,b,c
4,4,30008,c,c,c,c,False,c,c


## Identifier van het model gaan wij gebruiken om de bijbehorende informatie op te halen
wox -> wox uitleg

In [32]:
ee_nowox = pd.read_csv('../interpretations/final_model_EE_nowox_interpretations.csv')
ee_wox = pd.read_csv('../interpretations/final_model_EE_wox_interpretations.csv')
TK_nowox = pd.read_csv('../interpretations/final_model_TK_nowox_interpretations.csv')
TK_wox = pd.read_csv('../interpretations/final_model_TK_wox_interpretations.csv')

In [33]:
ee_nowox.head()

,Unnamed: 0,inw,oadh,inw65,KL,RLBRMTR,RLBWON,RLBBEV,RLBVRZ,RLBVEI,...,hh30inkHoog,aandeelMH19,auto30,auto60,ov30,ov60,fiets,banen,confidence,predicted_label
0,0,3.318658,3.930175,4.428645,3.443425,3.527892,3.172834,4.202292,4.160336,3.669563,...,3.432216,2.679482,3.920994,5.359560,3.087606,3.532189,1.900289,3.453048,0.550466,1
1,1,3.313034,3.634620,4.334825,3.378427,3.505774,4.382347,4.179473,3.935186,3.539427,...,3.449648,3.463188,3.931020,5.350854,2.851768,3.360046,1.714316,3.420658,0.683708,1
2,2,3.290712,3.840821,4.617866,3.403696,3.527690,4.907984,4.040775,3.953690,3.562878,...,3.398747,2.953709,3.879969,5.337527,2.737161,3.380181,1.861119,3.390353,0.512660,1
3,3,3.755752,2.654764,3.265404,3.474978,3.609885,2.946239,3.970471,3.998899,3.768880,...,3.436391,3.903928,4.061105,5.997788,3.234663,3.599758,2.185984,3.606681,0.643239,1
4,4,3.251035,3.213050,3.018885,3.431690,3.518267,4.181158,4.099533,3.375625,3.446905,...,3.408454,1.471394,3.782325,3.607321,3.521154,4.037943,3.797934,3.720797,0.561279,2


In [35]:
def capture_conf_ee(x):
    # check index
    index = x.ix[0]
    if x['wox_info']:
        return ee_wox.iloc[index]['confidence']
    else:
        return ee_nowox.iloc[index]['confidence']
capture_conf(predictions.iloc[1000])

def capture_conf_TK(x):
    # check index
    index = x.ix[0]
    if x['wox_info']:
        return TK_wox.iloc[index]['confidence']
    else:
        return TK_nowox.iloc[index]['confidence']
print(capture_conf_ee(predictions.iloc[1000]))
print(capture_conf_TK(predictions.iloc[1000]))

0.8096279000000001
0.82563585


In [36]:
predictions['TK_conf']  = predictions.apply(lambda x: capture_conf_TK(x), axis = 1)
predictions['EE_conf']  = predictions.apply(lambda x: capture_conf_ee(x), axis = 1)

In [37]:
predictions.head()

,Unnamed: 0,buurtcode,ee_wox,ee_nowox,TK_wox,TK_nowox,wox_info,EE_Ensemble,TK_Ensemble,TK_conf,EE_conf
0,0,30000,c,b,c,c,False,b,c,0.837450,0.550466
1,1,30001,c,b,c,c,False,b,c,0.843811,0.683708
2,2,30002,c,b,c,c,False,b,c,0.778277,0.512660
3,3,30007,c,b,c,c,False,b,c,0.944196,0.643239
4,4,30008,c,c,c,c,False,c,c,0.976197,0.561279


In [ ]:
#ee_wox_expl = pd.read_csv('../interpretations/')

In [77]:
# Capturing top 5 explanations

def capture_expl_ee(x):
    # check index
    index = x.ix[0]
    if x['wox_info']:
        # take top 5 from the dataframe
        temp = ee_wox.iloc[index].sort_values(ascending=False).drop(['Unnamed: 0', 'confidence'])[:5]
        temp_index, temp_values = [x for x in temp.index], [x for x in temp.values]
        # we zip so we can iterate
        return [f'{item[0]}-{item[1]:.1f}' for item in zip(temp_index,temp_values)]
    else:
        temp = ee_nowox.iloc[index].sort_values(ascending=False).drop(['Unnamed: 0', 'confidence'])[:5]
        temp_index, temp_values = [x for x in temp.index], [x for x in temp.values]
        return [f'{item[0]}-{item[1]:.1f}' for item in zip(temp_index,temp_values)]

def capture_expl_tk(x):
    # check index
    index = x.ix[0]
    if x['wox_info']:
        return TK_wox.iloc[index].sort_values(ascending=False).drop(['Unnamed: 0', 'confidence']).index[:5]
    else:
        return TK_nowox.iloc[index].sort_values(ascending=False).drop(['Unnamed: 0', 'confidence']).index[:5]
print(capture_expl_ee(predictions.iloc[1000]))
print(capture_expl_tk(predictions.iloc[1000]))

['woxww-11.4', 'woxm2-7.0', 'woxm2WG-5.0', 'AantalRest3-4.4', 'woxdelta1jr-3.2']
Index(['wox', 'fiets', 'ov30', 'AantalMusea10', 'AantalCafe3'], dtype='object')


In [49]:
temp = ee_wox.iloc[1000].sort_values(ascending=False).drop(['Unnamed: 0', 'confidence'])[:5]

In [62]:
temp_index = [x for x in temp.index]
temp_values = [x for x in temp.values]

In [63]:
temp.values

array([11.370603 ,  7.0356836,  4.977991 ,  4.396611 ,  3.2131932])

In [66]:
zipped = [x for x in zip(temp_index,temp_values)]

In [73]:
feature_list = []
for item in zipped:
    to_add = f'{item[0]}-{item[1]:.1f}'
    feature_list.append(to_add)

In [76]:
[f'{item[0]}-{item[1]:.1f}' for item in zip(temp_index,temp_values)]

['woxww-11.4',
 'woxm2-7.0',
 'woxm2WG-5.0',
 'AantalRest3-4.4',
 'woxdelta1jr-3.2']